## Day 14: AI-Powered Analytics: Genie & Mosaic AI

**Goal:**
1.  **Simulate Mosaic AI:** Use a Pre-trained LLM (Hugging Face Transformers) to perform Sentiment Analysis on customer feedback.
2.  **Simulate Genie:** Manually execute the "Natural Language" business questions using SQL.
3.  **Grand Finale:** Register the best model from Day 13 to the MLflow Model Registry.

**The Data:** I will generate some sample "Product Reviews" since the original dataset didn't have text.

In [0]:
# Install required libraries
# Installing the transformers library to use pre-trained AI models.
%pip install torch torchvision torchaudio transformers

# The following command restarts the kernel to load the library
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


##### Load necessary libraries

In [0]:
import pandas as pd
from transformers import pipeline
import mlflow

/local_disk0/.ephemeral_nfs/envs/pythonEnv-60e2d6c9-4c8c-425b-b3bb-9726b3723bfb/lib/python3.12/site-packages/torch/_vmap_internals.py:9: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  from torch.utils._pytree import _broadcast_to_and_flatten, tree_flatten, tree_unflatten


##### Data Exploration:
To check if our existing tables in the ecommerce.silver schema has any Text Reviews table

In [0]:
# List all tables in the Silver schema to see if there is a 'reviews' table
print("Tables in ecommerce.silver:")
display(spark.sql("SHOW TABLES IN ecommerce.silver"))

# Check the columns of the 'products' or 'orders' tables
# Looking for columns like "comment", "review", "description", or "body"
print("Checking 'gold_products' columns:")
display(spark.sql("DESCRIBE ecommerce.silver.gold_products"))

# 3. View the data to see if there are hidden text fields
print("Checking if there are any hidden text fields:")
display(spark.sql("SELECT * FROM ecommerce.silver.gold_products LIMIT 5"))

Tables in ecommerce.silver:


database,tableName,isTemporary
silver,cleaned_traffic,false
silver,gold_features,false
silver,optimized_traffic,false


Checking 'gold_products' columns:


{"ts": "2026-01-22 09:28:43.456", "level": "ERROR", "logger": "pyspark.sql.connect.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"[TABLE_OR_VIEW_NOT_FOUND] The table or view `ecommerce`.`silver`.`gold_products` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 1 pos 9;\n'DescribeRelation false, [col_name#13372, data_type#13373, comment#13374]\n+- 'UnresolvedTableOrView [ecommerce, silver, gold_products], DESCRIBE TABLE, true, true\n\"\n\tdebug_error_string = \"UNKNOWN:Error received from peer  {grpc_message:\"[TABLE_OR_VIEW_NOT_FOUND] The table or view

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7059960907268614>, line 8
      5 # Check the columns of the 'products' or 'orders' tables
      6 # Looking for columns like "comment", "review", "description", or "body"
      7 print("Checking 'gold_products' columns:")
----> 8 display(spark.sql("DESCRIBE ecommerce.silver.gold_products"))
     10 # 3. View the data to see if there are hidden text fields
     11 print("Checking if there are any hidden text fields:")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:875, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    872         _views.append(SubqueryAlias(df._plan, name))
    874 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 875 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    876 if "sql_command_result" in properties:
    877     df = Dat

##### Create Synthetic Reviews
Since our e-commerce data didn't have text reviews, Let's create a small dataset to analyze.

In [0]:
# Create a sample dataset of Customer Reviews
# Since real data is missing, we simulate it for the AI challenge
data = {
    "review_id": [1, 2, 3, 4, 5],
    "product_id": [101, 102, 101, 103, 102],
    "review_text": [
        "This product is amazing! I love the quality.",      # Positive
        "Terrible, it broke after two days. Waste of money.", # Negative
        "It's okay, does the job but shipping was slow.",     # Neutral/Negative
        "Absolutely fantastic! Best purchase ever.",          # Positive
        "Not what I expected, the color is wrong."            # Negative
    ],
    "category": ["Electronics", "Electronics", "Home", "Home", "Electronics"], # Added for Genie SQL
    "price": [500.0, 200.0, 50.0, 120.0, 200.0] # Added for Genie SQL
}

# 2. Convert to Spark DataFrame
df_reviews = spark.createDataFrame(pd.DataFrame(data))

# 3. Save as a temporary view so we can query it with SQL
df_reviews.createOrReplaceTempView("customer_reviews_gold")

print("Synthetic Data Created. You can now use 'customer_reviews_gold' for AI tasks.")
display(df_reviews)

Synthetic Data Created. You can now use 'customer_reviews_gold' for AI tasks.


review_id,product_id,review_text,category,price
1,101,This product is amazing! I love the quality.,Electronics,500.0
2,102,"Terrible, it broke after two days. Waste of money.",Electronics,200.0
3,101,"It's okay, does the job but shipping was slow.",Home,50.0
4,103,Absolutely fantastic! Best purchase ever.,Home,120.0
5,102,"Not what I expected, the color is wrong.",Electronics,200.0


##### Mosaic AI Simulation (Sentiment Analysis)
- Instead of training a model from scratch (which takes weeks), I used a **Pre-trained LLM** from Hugging Face.

- This is the core concept of **Mosaic AI**: taking off-the-shelf GenAI models and applying them to our data.

I stimulated the Mosaic AI by using a pre-trained "Transformer" model to read these reviews and determine if they are POSITIVE or NEGATIVE.

In [0]:

# Download a Pre-trained Sentiment Analysis Model
# This is a "DistilBERT" model fine-tuned for emotion/sentiment and which is fast and accurate for English text.
sentiment_pipeline = pipeline("sentiment-analysis")

# Define a function to apply the AI
def get_sentiment(text):
    # The model returns a list like [{'label': 'POSITIVE', 'score': 0.99}]
    result = sentiment_pipeline(text)[0]
    return result['label'], result['score']

# Get the data from Spark into Pandas for the AI model
# Note: In real Mosaic AI, a Spark UDF is used to do this in parallel
pdf_reviews = spark.table("customer_reviews_gold").toPandas()

print("The AI is reading the reviews...")

# Apply the AI model to every row
pdf_reviews[['sentiment', 'confidence']] = pdf_reviews['review_text'].apply(
    lambda x: pd.Series(get_sentiment(x))
)

# Log the "AI Experiment" to MLflow
with mlflow.start_run(run_name="Day_14_AI_Sentiment"):
    mlflow.log_param("model_type", "HuggingFace_DistilBERT")
    
    # Log average confidence (how sure was the AI?)
    avg_conf = pdf_reviews['confidence'].mean()
    mlflow.log_metric("avg_confidence", avg_conf)
    
    # Save the results
    result_spark = spark.createDataFrame(pdf_reviews)
    display(result_spark)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


The AI is reading the reviews...


review_id,product_id,review_text,category,price,sentiment,confidence
1,101,This product is amazing! I love the quality.,Electronics,500.0,POSITIVE,0.9998871088027954
2,102,"Terrible, it broke after two days. Waste of money.",Electronics,200.0,NEGATIVE,0.9998038411140442
3,101,"It's okay, does the job but shipping was slow.",Home,50.0,NEGATIVE,0.9980878233909607
4,103,Absolutely fantastic! Best purchase ever.,Home,120.0,POSITIVE,0.9998749494552612
5,102,"Not what I expected, the color is wrong.",Electronics,200.0,NEGATIVE,0.9995187520980835


### Genie Simulation (Natural Language to SQL)
Databricks Genie allows us to ask questions in English. Since this is a Community Edition, we will be acting as the "Genie" and write the SQL for the requested prompts.

**The Prompt:** "Show me total revenue by category."

#### Genie Query 1: 
"Show me total revenue by category"

In [0]:
%sql
SELECT 
    category, 
    SUM(price) as total_revenue
FROM customer_reviews_gold
GROUP BY category
ORDER BY total_revenue DESC

category,total_revenue
Electronics,900.0
Home,170.0


#### Genie Query 2:
"Which products have the highest conversion rate?"

In [0]:
%sql
-- Highest "Conversion" (Sales Count) by Product --
SELECT 
    product_id,
    COUNT(*) as total_sales
FROM customer_reviews_gold
GROUP BY product_id
ORDER BY total_sales DESC
LIMIT 5

product_id,total_sales
101,2
102,2
103,1


#### Genie Query 3:
"What's the trend of daily purchases over time?"

In [0]:
%sql
-- Purchase Trend over Time (Hour of Day) --
SELECT 
    hour,
    COUNT(*) as total_purchases
FROM ecommerce.silver.gold_features
WHERE is_purchased = 1
GROUP BY hour
ORDER BY hour ASC

hour,total_purchases
0,5771
1,9741
2,25124
3,55968
4,85232
5,101428
6,109424
7,112108
8,120453
9,126615


#### Genie Query 4:
"Find customers who viewed but never purchased"

In [0]:
%sql
-- Non-Buyers (Viewed but didn't buy) --
SELECT 
    count(*) as count_of_non_buyers
FROM ecommerce.silver.gold_features
WHERE is_purchased = 0

count_of_non_buyers
107856765


#### Genie Query 5: 
"Find customers who are unhappy (Negative Sentiment)."

In [0]:
# Filter the AI results for Negative reviews
print("Negative Reviews:")
display(result_spark.filter("sentiment = 'NEGATIVE'"))

Negative Reviews:


review_id,product_id,review_text,category,price,sentiment,confidence
2,102,"Terrible, it broke after two days. Waste of money.",Electronics,200.0,NEGATIVE,0.9998038411140442
3,101,"It's okay, does the job but shipping was slow.",Home,50.0,NEGATIVE,0.9980878233909607
5,102,"Not what I expected, the color is wrong.",Electronics,200.0,NEGATIVE,0.9995187520980835


## Model Registry
To finish the 14-Day Challenge, we will take our **Best Model** from Day 13 (The Balanced Logistic Regression) and register it as "Production Ready."

In [0]:
# Best Run ID from Day 13
best_run_id = "d95eb9431a4b441bbb152bbdd135b0af" 
model_name = "Final_Purchase_Predictor_v1"

# Construct the model URI
model_uri = f"runs:/{best_run_id}/model"

# Register the model
print(f"Registering Run ID: {best_run_id}...")
mlflow.register_model(model_uri, model_name)

print(f"Model registered successfully as '{model_name}'!")

Registering Run ID: d95eb9431a4b441bbb152bbdd135b0af...


Registered model 'Final_Purchase_Predictor_v1' already exists. Creating a new version of this model...


---------------------------------------------------------------------------
MlflowException                           Traceback (most recent call last)
File <command-4665204095140393>, line 10
      8 # Register the model
      9 print(f"Registering Run ID: {best_run_id}...")
---> 10 mlflow.register_model(model_uri, model_name)
     12 print(f"Model registered successfully as '{model_name}'!")

File /databricks/python/lib/python3.12/site-packages/mlflow/tracking/_model_registry/fluent.py:80, in register_model(model_uri, name, await_registration_for, tags)
     20 def register_model(
     21     model_uri,
     22     name,
   (...)
     25     tags: Optional[dict[str, Any]] = None,
     26 ) -> ModelVersion:
     27     """Create a new model version in model registry for the model files specified by ``model_uri``.
     28 
     29     Note that this method assumes the model registry backend URI is the same as that of the
   (...)
     78         Version: 1
     79     """
---> 80     r

- The error happens because in Databricks Unity Catalog, we cannot register a "Mystery Model".

- The inputs and outputs (called a signature) must be defined so the system knows how to use it safely.

- Since the Day 13 model didn't have a signature, I followed the following steps:

  1. Load the model from Day 13.
  2. Add the Signature (using the data to show what inputs look like).
  3. Re-log & Register it as the final version.

In [0]:
from mlflow.models.signature import infer_signature

# Setup: Day 13 Run ID (The one that failed)
bad_run_id = "d95eb9431a4b441bbb152bbdd135b0af"

print(f"Loading model from Run {bad_run_id} to fix Signature...")

# Load the model from the old run
loaded_model = mlflow.sklearn.load_model(f"runs:/{bad_run_id}/model")

# Create a Temperory Input to define the Signature
# (The model expects: price_log, hour, is_weekend)
temp_data = pd.DataFrame({
    "price_log": [4.5],
    "hour": [14.0],
    "is_weekend": [0.0]
})
prediction = loaded_model.predict(temp_data)

# Create the Signature (Input Schema -> Output Schema)
signature = infer_signature(temp_data, prediction)

# Re-log the model WITH the Signature (The "Production" Run)
with mlflow.start_run(run_name="Day_14_Final_Production") as run:
    mlflow.sklearn.log_model(
        loaded_model, 
        "model", 
        signature=signature,
        input_example=temp_data
    )
    
    # Get the NEW Run ID
    new_run_id = run.info.run_id
    print(f"Model re-logged with Signature! New Run ID: {new_run_id}")

    # Register the Valid Model
    model_name = "Final_Purchase_Predictor_v1"
    model_uri = f"runs:/{new_run_id}/model"
    
    mlflow.register_model(model_uri, model_name)
    print(f"Model successfully registered as '{model_name}' in Unity Catalog.")

Loading model from Run d95eb9431a4b441bbb152bbdd135b0af to fix Signature...


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Model re-logged with Signature! New Run ID: 00dc2a4cf3ba4732adb6b8d1f1962a82


Registered model 'Final_Purchase_Predictor_v1' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Model successfully registered as 'Final_Purchase_Predictor_v1' in Unity Catalog.


Created version '3' of model 'workspace.default.final_purchase_predictor_v1'.
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


## Testing the AI model to see how smart it is!

In [0]:
# Test the AI on BRAND NEW data it has never seen before
my_custom_reviews = [
    "The delivery was super fast, but the product feels cheap.",  # Mixed/Negative?
    "I am never buying from this brand again, absolute garbage.", # Strong Negative
    "Omg this is the best thing ever!!!",                         # Strong Positive
    "It is okay, nothing special."                                # Neutral/Negative
]

print("Testing on NEW Unseen Reviews:\n")

for review in my_custom_reviews:
    label, score = get_sentiment(review)
    
    # Print the result
    print(f"Review: '{review}'")
    print(f"Verdict: {label} (Confidence: {score:.4f})\n")

Testing on NEW Unseen Reviews:

Review: 'The delivery was super fast, but the product feels cheap.'
Verdict: NEGATIVE (Confidence: 0.9996)

Review: 'I am never buying from this brand again, absolute garbage.'
Verdict: NEGATIVE (Confidence: 0.9988)

Review: 'Omg this is the best thing ever!!!'
Verdict: POSITIVE (Confidence: 0.9999)

Review: 'It is okay, nothing special.'
Verdict: NEGATIVE (Confidence: 0.8283)

